In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from PIL import Image
import scipy.ndimage
from tqdm import tqdm

In [5]:
class FluidSimulation:
    def __init__(self, image_path, config=None):
        self.config = {
            'dt': 0.1,             # time step
            'diffusion': 0.00005,  # Diffusion coefficient
            'viscosity': 0.00001,  # Viscosity
            'iterations': 20,      # Num iterations for diffusion
            'scale': 4,            # downsampling
            'frames': 200          # Frame number
        }
        
        if config:
            self.config.update(config)
            
        self.load_image(image_path)
        
        self.vx = np.zeros_like(self.density)
        self.vy = np.zeros_like(self.density)
        self.vx_prev = np.zeros_like(self.density)
        self.vy_prev = np.zeros_like(self.density)
        self.p = np.zeros_like(self.density)
        self.div = np.zeros_like(self.density)
        
        self.fig, self.ax = plt.subplots(figsize=(10, 10))
        self.img_plot = None
        self.frame = 0
        
    def load_image(self, image_path):
        img = Image.open(image_path)
        img_gray = img.convert('L')
        scale = self.config['scale']
        if scale > 1:
            new_size = (img.width // scale, img.height // scale)
            img_gray = img_gray.resize(new_size, Image.LANCZOS)
        
        self.density = np.array(img_gray).astype(float) / 255.0
        self.density_prev = self.density.copy()
        
        self.height, self.width = self.density.shape
        self.original_img = np.array(img)

    def diffuse(self, b, x, x0, diff, dt):
        a = dt * diff * self.width * self.height
        
        for _ in range(self.config['iterations']):
            x[1:-1, 1:-1] = (x0[1:-1, 1:-1] + a * (
                x[2:, 1:-1] + x[:-2, 1:-1] + 
                x[1:-1, 2:] + x[1:-1, :-2]
            )) / (1 + 4 * a)
            
            self.set_boundary(b, x)
    
    def project(self, vx, vy, p, div):
        div[1:-1, 1:-1] = -0.5 * (
            vx[2:, 1:-1] - vx[:-2, 1:-1] + 
            vy[1:-1, 2:] - vy[1:-1, :-2]
        ) / (self.width * self.height)
        
        p.fill(0)
        
        self.set_boundary(0, div)
        self.set_boundary(0, p)
        
        for _ in range(self.config['iterations']):
            p[1:-1, 1:-1] = (div[1:-1, 1:-1] + 
                p[2:, 1:-1] + p[:-2, 1:-1] + 
                p[1:-1, 2:] + p[1:-1, :-2]) / 4
            
            self.set_boundary(0, p)
        
        vx[1:-1, 1:-1] -= 0.5 * (p[2:, 1:-1] - p[:-2, 1:-1]) * self.width
        vy[1:-1, 1:-1] -= 0.5 * (p[1:-1, 2:] - p[1:-1, :-2]) * self.height
        
        self.set_boundary(1, vx)
        self.set_boundary(2, vy)
    
    def advect(self, b, d, d0, vx, vy, dt):
        dt0x = dt * self.width
        dt0y = dt * self.height
        
        y, x = np.mgrid[0:self.height, 0:self.width]
        x_prev = np.clip(x - dt0x * vx, 0.5, self.width - 1.5)
        y_prev = np.clip(y - dt0y * vy, 0.5, self.height - 1.5)

        x0 = np.floor(x_prev).astype(int)
        y0 = np.floor(y_prev).astype(int)
        x1 = x0 + 1
        y1 = y0 + 1
        
        s1 = x_prev - x0
        s0 = 1 - s1
        t1 = y_prev - y0
        t0 = 1 - t1
        x0 = np.clip(x0, 0, self.width - 1)
        x1 = np.clip(x1, 0, self.width - 1)
        y0 = np.clip(y0, 0, self.height - 1)
        y1 = np.clip(y1, 0, self.height - 1)
        
        d[:] = (s0 * (t0 * d0[y0, x0] + t1 * d0[y1, x0]) +
                s1 * (t0 * d0[y0, x1] + t1 * d0[y1, x1]))
        
        self.set_boundary(b, d)
    
    def set_boundary(self, b, x):

        if b == 2:  # For vy
            x[0, :] = -x[1, :]
            x[-1, :] = -x[-2, :]
        else:
            x[0, :] = x[1, :]
            x[-1, :] = x[-2, :]
            
        if b == 1:  # For vx
            x[:, 0] = -x[:, 1]
            x[:, -1] = -x[:, -2]
        else:
            x[:, 0] = x[:, 1]
            x[:, -1] = x[:, -2]
            
        x[0, 0] = 0.5 * (x[1, 0] + x[0, 1])
        x[0, -1] = 0.5 * (x[1, -1] + x[0, -2])
        x[-1, 0] = 0.5 * (x[-2, 0] + x[-1, 1])
        x[-1, -1] = 0.5 * (x[-2, -1] + x[-1, -2])
    
    def add_random_forces(self):
        for _ in range(3):
            x = np.random.randint(5, self.width - 5)
            y = np.random.randint(5, self.height - 5)
            dx = (np.random.random() - 0.5) * 0.2
            dy = (np.random.random() - 0.5) * 0.2
            
            sigma = 3.0
            y_indices, x_indices = np.mgrid[0:self.height, 0:self.width]
            gaussian = np.exp(-((x_indices - x)**2 + (y_indices - y)**2) / (2 * sigma**2))
            
            self.vx += gaussian * dx
            self.vy += gaussian * dy
    
    def step(self):
        dt = self.config['dt']
        visc = self.config['viscosity']
        diff = self.config['diffusion']
        
        self.add_random_forces()
        
        self.diffuse(1, self.vx_prev, self.vx, visc, dt)
        self.diffuse(2, self.vy_prev, self.vy, visc, dt)
        
        self.project(self.vx_prev, self.vy_prev, self.p, self.div)
    
        self.advect(1, self.vx, self.vx_prev, self.vx_prev, self.vy_prev, dt)
        self.advect(2, self.vy, self.vy_prev, self.vx_prev, self.vy_prev, dt)
        
        self.project(self.vx, self.vy, self.p, self.div)
        
        self.diffuse(0, self.density_prev, self.density, diff, dt)
        
        self.advect(0, self.density, self.density_prev, self.vx, self.vy, dt)
    
    def update(self, frame):
        self.step()
        
        if self.img_plot is None:
            self.img_plot = self.ax.imshow(self.density, cmap='gray')
            self.ax.set_title(f'Frame {self.frame}')
            plt.colorbar(self.img_plot, ax=self.ax)
        else:
            self.img_plot.set_array(self.density)
            self.ax.set_title(f't= {self.frame} s')
        
        self.frame += 1
        plt.close()
        return [self.img_plot]
    
    def animate(self):
        anim = FuncAnimation(
            self.fig, 
            self.update, 
            frames=self.config['frames'],
            interval=50,
            blit=True
        )
        plt.show()
        return anim
    
    def save_animation(self, filename='fluid_simulation.mp4'):
        anim = FuncAnimation(
            self.fig, 
            self.update, 
            frames=self.config['frames'],
            interval=50,
            blit=True
        )
        anim.save(filename, writer='ffmpeg', fps=20, dpi=100)
        return anim


if __name__ == "__main__":
    image_path = "./charbel.png" #replace this with your image path
    
    config = {
        'dt': 0.1,
        'diffusion': 0.00005,
        'viscosity': 0.00001,
        'iterations': 50,
        'scale': 4,
        'frames': 1000
    }
    
    
    sim = FluidSimulation(image_path, config)
    sim.save_animation('simulation.mp4')